# Data Wrangling with ``pyarrow``

## Index 和 Slice 的概念及用途

**概念**

pyarrow 和 pandas 都有类似 index 的概念. 所谓 index 就是行号, 给每行的一个标识. 在数据处理中 ``RangeIndex`` 是最常用的也是性能最好的 index. ``RangeIndex`` 的意思是给行自动从 0, 1, 2, ... 开始给定序号. 这样做可以无需真的在内存中维护 index 的数据, 而只要在 metadata 中记录我们用的是 ``RangeIndex`` 即可, 从而节约内存.

slice 的概念是对一个 array 进行切片, 这和 python 中的 list slicing 很像. 既然要切片就要有一个规定怎么切. 那么 index 就是这个规定怎么切的标记. 例如你有个 array ``["a", "b", "c", "d"]``, 又有一个 index ``[3, 2, 1, 0]``, 那么切出来就是 ``["d", "c", "b", "a"]``. 使用 slice 操作还有一个好处是不会对数据进行拷贝, 只是基于 index 创建一个新的 view. 

**用途**

在数据处理中, 选取符合条件的部分行, 对行进行排序, 等各种操作都是通过 index 和 slice 进行的. 

- 比如你要对某一列中的所有 null 的值给予一个默认值, 这个操作的本质是: 先找到所有 null 值的位置的 index, 然后根据 index slice 一下, 给这些内存地址赋值.
- 比如你要根据某一列的值对所有行排序, 这个操作的本质是: 对某列的值进行排序, 获得按序排列的 index, 然后根据 index slice 一下, 给 table 创建一个新的 view.
- 比如你要选取符合条件的不分行, 这个操作的本质是: 对列中的值根据条件进行筛选, 获得符合条件的行的 index, 然后根据 index slice 一下, 给 table 创建一个新的 view.

所以 Index 和 Slice 对于数据处理是非常重要的.

In [2]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.compute as pc

## Sort by value

在 pyarrow 中, 和 pandas 类似, 对行的访问也是通过 index 的. 你想要 基于某个(或多个)列的值 对行进行排序. 这个行为的本质是对列的值进行排序, 获得一个 indice, 然后用这个 indice 去 slice 原来的 Table, 获得一个 view (不进行数据拷贝). 这里有两个关键的 API: ``pyarrow.compute.sort_indices``, 获得一个 基于某个(或多个)列的值 对行进行排序 的 indice. ``pyarrow.compute.take``, 用 indice 对 array liked object 进行 slice, 可以是 array, 也可以是 Table.

Ref:

- https://arrow.apache.org/docs/python/generated/pyarrow.compute.take.html#pyarrow.compute.take
- https://arrow.apache.org/docs/python/generated/pyarrow.compute.sort_indices.html#pyarrow.compute.sort_indices

**根据一列排序**

In [2]:
t = pa.table({
    "account_id": [1, 2, 3, 4, 5, 6],
    "account_type": ["checking", "saving", "checking", "saving", "checking", "saving"],
    "balance": [57, 62, 23, 18, 79, 94]
})
t.to_pandas()

,account_id,account_type,balance
0,1,checking,57
1,2,saving,62
2,3,checking,23
3,4,saving,18
4,5,checking,79
5,6,saving,94


In [3]:
indice = pc.sort_indices(t, sort_keys=[("balance", "ascending")])
indice

[
  3,
  2,
  0,
  1,
  4,
  5
]

In [4]:
t_sorted_by_balance = pc.take(t, indice)
t_sorted_by_balance.to_pandas()

,account_id,account_type,balance
0,4,saving,18
1,3,checking,23
2,1,checking,57
3,2,saving,62
4,5,checking,79
5,6,saving,94


**根据多列排序**

In [5]:
pc.take(
    t, 
    pc.sort_indices(
        t, 
        sort_keys=[
            ("account_type", "ascending"), 
            ("balance", "ascending"), 
        ]
    ),
).to_pandas()

,account_id,account_type,balance
0,3,checking,23
1,1,checking,57
2,5,checking,79
3,4,saving,18
4,2,saving,62
5,6,saving,94


## Filter

Filter 的关键就是先用 比较 / 测试 函数获得 logic index, 然后对多个 index 用 逻辑运算 函数进行排列组合, 最后用 ``pyarrow.compute.filter`` slice 出一个 view 即可.

Ref:

- https://arrow.apache.org/docs/python/api/compute.html#comparisons
- https://arrow.apache.org/docs/python/api/compute.html#containment-tests
- https://arrow.apache.org/docs/python/api/compute.html#logical-functions
- https://arrow.apache.org/docs/python/generated/pyarrow.compute.filter.html#pyarrow.compute.filter

In [6]:
t = pa.table({
    "account_id": [1, 2, 3, 4, 5, 6],
    "account_type": ["checking", "saving", "checking", "saving", "checking", "saving"],
    "balance": [57, 62, 23, 18, 79, 94]
})
t.to_pandas()

,account_id,account_type,balance
0,1,checking,57
1,2,saving,62
2,3,checking,23
3,4,saving,18
4,5,checking,79
5,6,saving,94


In [7]:
# 根据单个条件筛选
pc.filter(
    t, 
    pc.greater(t["balance"], 50),
).to_pandas()

,account_id,account_type,balance
0,1,checking,57
1,2,saving,62
2,5,checking,79
3,6,saving,94


In [8]:
# 根据多个条件筛选
pc.filter(
    t, 
    pc.and_(
        pc.equal(t["account_type"], "checking"),
        pc.greater(t["balance"], 50),
    )
).to_pandas()

,account_id,account_type,balance
0,1,checking,57
1,5,checking,79


## Group By 分拆

- ``pyarrow.Table.group_by`` 是在 7.0.0 之后加入的. 之前要用 ``pyarrow.TableGroupBy``

In [10]:
n_rows = 1000

a_enum = ["a1", "a2", "a3"]
b_enum = ["b1", "b2", "b3"]

t = pa.table({
    "a": pa.DictionaryArray.from_arrays(
        np.random.randint(low=0, high=len(a_enum), size=n_rows), 
        a_enum,
    ),
    "b": pa.DictionaryArray.from_arrays(
        np.random.randint(low=0, high=len(b_enum), size=n_rows), 
        b_enum,
    ),
    "v": np.random.randint(100, size=n_rows)
})
t.to_pandas().head(3)

,a,b,v
0,a2,b3,32
1,a2,b3,85
2,a2,b1,22


In [3]:
for i in pa.TableGroupBy(t, ["a"]):
    print(i)

AttributeError: module 'pyarrow' has no attribute 'TableGroupBy'